# 0.0. Info Table

Monster Hunter 2 Dos Database App

    1. Lista de Quests separadas por Ranks.
    2. Lista de Monstros e suas caracteristicas.
    3. Lista de Kitchen Combination separadas por season.

In [24]:
import re
import sqlite3
import requests
import pandas as pd

from bs4 import BeautifulSoup
from sqlalchemy import create_engine

# 1.0. Monster Data

## 1.1. Monster Data Collect

In [144]:
url = 'https://monsterhunter.fandom.com/wiki/MH2:_Monsters'
hdr = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5),AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

soup = BeautifulSoup( requests.get( url, hdr ).text, 'html.parser' )

In [145]:
# List Monster Names
list_monsters  = [p.get_text() for p in soup.find_all('b')][:-12]
list_monsters1 = [p.replace(' ', '_') for p in list_monsters] 

In [147]:
df_monsters = pd.DataFrame( columns=['jp_name', 'class', 'element', 'weak', 'gen'] )

for i in range( len( list_monsters1 ) ):
    url = 'https://monsterhunter.fandom.com/wiki/' + list_monsters1[i]
    soup = BeautifulSoup( requests.get( url, hdr ).text, 'html.parser' )
    print(url)
    
    # jp_name, class, element, weakness, gen
    jp_name    = re.match( '\w+' , soup.find('aside').find_all('h2')[1].get_text() ).group(0)
    m_class    = soup.find('aside').find_all('section', class_='pi-item pi-group pi-border-color')[1].find_all('a')[1].get_text()
    m_element  = soup.find('aside').find_all('section', class_='pi-item pi-group pi-border-color')[1].find_all('div', class_='pi-data-value pi-font')[1].get_text().replace(' ', '')
    m_gen      = soup.find_all('section', class_='pi-item pi-group pi-border-color')[1].find_all( 'div', class_='pi-data-value pi-font' )[-1].get_text()
    m_weakness = [p.get_text().replace(' ', '') for p in 
                          soup.find_all('section', class_='pi-item pi-group pi-border-color')[1]
                              .find_all('div', class_='pi-data-value pi-font')[3]
                              .find_all('small')][:2]
    
    if len( m_weakness ) == 0:
        m_weakness = 'None'
    
    if len( m_weakness ) == 2:
        m_weakness = m_weakness[0] + ', ' + m_weakness[1]
        
    else:
        m_weakness = m_weakness[0]
    
    # Generate DataFrame
    aux = pd.DataFrame( [jp_name, m_class, m_element, m_weakness, m_gen] ).T
    aux.columns = ['jp_name', 'class', 'element', 'weak', 'gen']
    
    # DataFrame with all Monsters
    df_monsters = pd.concat( [df_monsters, aux], axis=0 ) 

https://monsterhunter.fandom.com/wiki/Felyne
https://monsterhunter.fandom.com/wiki/Melynx
https://monsterhunter.fandom.com/wiki/Shakalaka
https://monsterhunter.fandom.com/wiki/Vespoid
https://monsterhunter.fandom.com/wiki/Hornetaur
https://monsterhunter.fandom.com/wiki/Great_Thunderbug
https://monsterhunter.fandom.com/wiki/Anteka
https://monsterhunter.fandom.com/wiki/Popo
https://monsterhunter.fandom.com/wiki/Kelbi
https://monsterhunter.fandom.com/wiki/Mosswine
https://monsterhunter.fandom.com/wiki/Aptonoth
https://monsterhunter.fandom.com/wiki/Apceros
https://monsterhunter.fandom.com/wiki/Velociprey
https://monsterhunter.fandom.com/wiki/Giaprey
https://monsterhunter.fandom.com/wiki/Genprey
https://monsterhunter.fandom.com/wiki/Ioprey
https://monsterhunter.fandom.com/wiki/Remobra
https://monsterhunter.fandom.com/wiki/Cephalos
https://monsterhunter.fandom.com/wiki/Hermitaur
https://monsterhunter.fandom.com/wiki/Ceanataur
https://monsterhunter.fandom.com/wiki/Bullfango
https://monsterhun

## 1.2. Monster Data Cleaning

In [199]:
df = df_monsters

# First Cleaning and english names
df = df.reset_index( drop=True )
df['name'] = list_monsters
df = df[['name', 'jp_name', 'class', 'element', 'weak', 'gen']]

# Individual Columns
df['gen'] = df['gen'].apply( lambda x: x.replace('*', ''))

df.loc[df['weak'].str.contains('N'), 'weak']   = 'None'
df.loc[df['weak'].str.contains('2nd'), 'weak'] = 'Ice, Dragon'

df['element'] = df['element'].apply( lambda x: x.replace('N/A', 'None'))
df['element'] = df['element'].str.extract('([a-zA-Z ]+)')
df.loc[df['element'] == 'Earth', 'element'] = 'None'
df.loc[df['element'] == 'FireDragon', 'element'] = 'Dragon'
df.loc[df['element'] == 'ThunderDragon', 'element'] = 'Dragon'
df.loc[df['element'] == 'DragonFireBlackFlame', 'element'] = 'Dragon'
df.loc[df['element'] == 'DragonFireCrimsonDemon', 'element'] = 'Dragon'

df = df.reset_index( drop=True )

df.head()

,name,jp_name,class,element,weak,gen
0,Felyne,アイルー,Lynian,None,"Fire, Water",First
1,Melynx,メラルー,Lynian,None,"Fire, Water",First
2,Shakalaka,チャチャブー,Lynian,None,Ice,Second
3,Vespoid,ランゴスタ,Neopteron,None,"Fire, Poison",First
4,Hornetaur,カンタロス,Neopteron,None,"Fire, Water",First


## 1.3. Monster Data Storange

In [204]:
con = sqlite3.connect( '../sql_databases/monsters.sqlite' )
c   = con.cursor()

query_create_table = '''
    CREATE TABLE monsters (
        name       TEXT,
        jp_name    TEXT,
        class      TEXT,
        element    TEXT,
        weak       TEXT,
        gen        TEXT );'''

c.execute( query_create_table )
con.commit()

con = create_engine( 'sqlite:///../sql_databases/monsters.sqlite' )
df.to_sql( 'monsters', con=con, if_exists='append', index=False )